# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [39]:
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

### **Extract**

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.


In [40]:
import pandas as pd
df = pd.read_csv('content/SDW2023.csv')
user_ids = df['UserID'].tolist()
user_ids

[13, 24, 14]

In [41]:
#Obtendo os dados de cada ID usando a API
import requests #Responsável pela interface de interação com chamadas http
import json

def get_user(id):
    response = requests.get(f'{sdw2023_api_url}/users/{id}')
    return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 13,
    "name": "anguile",
    "account": {
      "id": 20,
      "number": "00010-8",
      "agency": "2003",
      "balance": 500.0,
      "limit": 1000.0
    },
    "card": {
      "id": 13,
      "number": "**** **** **** 0112",
      "limit": 500.0
    },
    "features": [
      {
        "id": 10,
        "icon": "string",
        "description": "string"
      }
    ],
    "news": [
      {
        "id": 24,
        "icon": "string",
        "description": "string"
      },
      {
        "id": 122,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/insurance.svg",
        "description": "Invista seu dinheiro com sabedoria e garanta um futuro promissor."
      },
      {
        "id": 250,
        "icon": "https://images.genius.com/31fac7e2bef0ff83f25dbd8077863794.1000x1000x1.jpg",
        "description": "Eduardo esteve aqui"
      },
      {
        "id": 251,
        "icon": "https://images.genius.com/31fac7e2bef0ff83f25db

## **Transform**

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [1]:
#Integração com a OpenAI !pip install openai
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
openai_api_key = '******************'

In [43]:
#TODO implementar a integração com o ChatGPT usando o modelo GPT-4
import openai

openai.api_key = openai_api_key

def generate_ai_news(user): 
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
    {
        "role": "system", 
        "content": "Você é um tecnico explosivista com experiência em EOD."
    },
    {
        "role": "user", 
        "content": f"Crie técnicas de desativação para {user['name']} com no maximo 100 caracteres" }
      ]
    )
    return completion.choices[0].message.content.strip('\"')

for user in users:
    news = generate_ai_news(user)
    print(news)
    user['news'].append({
        "icon":"None",
        "description": news
    })


1. Corte o fio vermelho. 
2. Remova a bateria. 
3. Desconecte a antena. 
4. Desarme o detonador.
1. Remova bateria. 
2. Desconecte fios. 
3. Corte circuito. 
4. Desarme detonador. 
5. Isole explosivo.
1. Remoção do detonador com pinça. 
2. Neutralização do explosivo com água. 
3. Corte do fio de ativação com tesoura. 
4. Desativação controlada com robô especializado. 
5. Uso de explosivo contra explosivo (contra-carga).


## **Load**

In [44]:
#Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.
def update_user(user):
    response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
    return True if response.status_code == 200 else False
    
for user in users:
    success = update_user(user)
    print(f"User {user['name']} update? {success}")

User anguile update? True
User Mar update? True
User Maria update? True
